# Generate CGR images using GAN

## GAN model

In [8]:
import os
import numpy as np
from numpy import zeros
from numpy import expand_dims
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from numpy.random import randint
from matplotlib import pyplot
import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Sequential
from keras.optimizers.legacy import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Conv2DTranspose

# define the standalone discriminator model
def define_discriminator(in_shape=(128,128,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(256, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(512, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(512, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    #model.add(Conv2D(1024, (3,3), strides=(2,2), padding='same'))
    #model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 4x4 image
    n_nodes = 512 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 512)))
    # upsample to 8x8
    model.add(Conv2DTranspose(512, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(Conv2DTranspose(512, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 64x64
    model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 128x128
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer
    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

def getAllSamples(classID):
    folderName = trainPath + '/' + classNames[classID]
    arrays = []
    imageSamples = os.listdir(folderName)
    print(imageSamples)
    for i in range(0, len(imageSamples)):
        img = load_img(folderName + '/' + imageSamples[i])
        #print('Poza {} este {}'.format(i, imageSamples[i]))
        # convert to numpy array
        img_array = img_to_array(img)
        img_array = img_array.astype('uint8')
        arrays.append(img_array)
    return np.stack((arrays))

def load_real_samples(classID):
    # load cifar10 dataset
    trainX = getAllSamples(classID)
    # convert from unsigned ints to floats
    X = trainX.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    print('dataset.shape[0] = {}, n_samples = {}'.format(dataset.shape[0], n_samples))
    ix = randint(0, dataset.shape[0], n_samples)
    #print('ix = ', ix)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

# generate n fake samples with class labels
def generate_fake_samples(n_samples):
    # generate uniform random numbers in [0,1]
    X = rand(128 * 128 * 3 * n_samples)
    # update to have the range [-1, 1]
    X = -1 + X * 2
    # reshape into a batch of color images
    X = X.reshape((n_samples, 128, 128, 3))
    # generate 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    #print('x_input shape = ', x_input.shape)
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i])
    # save plot to file
    filename = outputFolder + 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150, classID=0):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    filename = outputFolder + 'generator_class%d_model_%03d.h5' % (classID, epoch+1)
    g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=50, n_batch=32, classID=0):
    print('n_batch = ', n_batch)
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):

            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            #print('shape = ', X_real.shape)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) > n_epochs - 30:
            n_samples=10
            summarize_performance(i, g_model, d_model, dataset, latent_dim, n_samples, classID)

mainPath = '/content/drive/MyDrive/ML_Dataset/CCAT10'
trainPath = mainPath + '/train'

classNames = os.listdir(trainPath)
print(classNames)
for classID in range(0, 10):
#classID = 8
  print('Creating models for class ', classID)
  outputFolder = '/content/drive/MyDrive/ML_Dataset/AugmentedModels/' + classNames[classID] + '/'

  try:
      os.makedirs(outputFolder)
  except OSError:
      pass

  n_epochs=200
  n_batch=8
  # size of the latent space
  latent_dim = 100
  # create the discriminator
  d_model = define_discriminator()
  # create the generator
  g_model = define_generator(latent_dim)
  # create the gan
  gan_model = define_gan(g_model, d_model)
  # load image data
  dataset = load_real_samples(classID)

  # train model
  train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs, n_batch, classID)
  keras.backend.clear_session()

['AlanCrosby', 'AlexanderSmith', 'BenjaminKangLim', 'DavidLawder', 'JimGilchrist', 'JohnMastrini', 'MarcelMichelson', 'MureDickie', 'RobinSidel', 'ToddNissen']
Creating models for class  0
['AlanCrosby2_Train.jpg', 'AlanCrosby8_Train.jpg', 'AlanCrosby1_Train.jpg', 'AlanCrosby10_Train.jpg', 'AlanCrosby3_Train.jpg', 'AlanCrosby6_Train.jpg', 'AlanCrosby9_Train.jpg', 'AlanCrosby7_Train.jpg', 'AlanCrosby4_Train.jpg', 'AlanCrosby5_Train.jpg', 'AlanCrosby11_Train.jpg', 'AlanCrosby23_Train.jpg', 'AlanCrosby12_Train.jpg', 'AlanCrosby15_Train.jpg', 'AlanCrosby25_Train.jpg', 'AlanCrosby28_Train.jpg', 'AlanCrosby24_Train.jpg', 'AlanCrosby14_Train.jpg', 'AlanCrosby26_Train.jpg', 'AlanCrosby22_Train.jpg', 'AlanCrosby16_Train.jpg', 'AlanCrosby18_Train.jpg', 'AlanCrosby21_Train.jpg', 'AlanCrosby19_Train.jpg', 'AlanCrosby17_Train.jpg', 'AlanCrosby27_Train.jpg', 'AlanCrosby20_Train.jpg', 'AlanCrosby13_Train.jpg', 'AlanCrosby29_Train.jpg', 'AlanCrosby41_Train.jpg', 'AlanCrosby44_Train.jpg', 'AlanCrosby31

>Accuracy real: 80%, fake: 100%
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 17ms/step
>172, 1/6, d1=0.020, d2=0.168 g=5.396
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 20ms/step
>172, 2/6, d1=0.408, d2=1.670 g=20.154
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 17ms/step
>172, 3/6, d1=3.528, d2=0.000 g=7.845
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 18ms/step
>172, 4/6, d1=0.006, d2=0.050 g=3.816
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 18ms/step
>172, 5/6, d1=0.003, d2=0.546 g=7.856
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 17ms/step
>172, 6/6, d1=0.004, d2=0.000 g=10.193
dataset.shape[0] = 50, n_samples = 10
1/1 [==============================] - 0s 29ms/step


>Accuracy real: 50%, fake: 100%
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 26ms/step
>173, 1/6, d1=0.735, d2=0.189 g=3.347
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 29ms/step
>173, 2/6, d1=0.005, d2=0.793 g=10.061
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 31ms/step
>173, 3/6, d1=0.048, d2=0.000 g=11.359
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 32ms/step
>173, 4/6, d1=0.431, d2=0.007 g=4.614
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 21ms/step
>173, 5/6, d1=0.050, d2=1.085 g=10.184
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 19ms/step
>173, 6/6, d1=0.145, d2=0.000 g=13.032
dataset.shape[0] = 50, n_samples = 10
1/1 [==============================] - 0s 19ms/step


>Accuracy real: 70%, fake: 100%
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 29ms/step
>174, 1/6, d1=0.648, d2=0.004 g=5.195
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 19ms/step
>174, 2/6, d1=0.011, d2=0.812 g=8.802
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 19ms/step
>174, 3/6, d1=0.016, d2=0.000 g=10.257
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 20ms/step
>174, 4/6, d1=1.770, d2=0.098 g=2.632
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 23ms/step
>174, 5/6, d1=0.002, d2=1.737 g=10.995
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 20ms/step
>174, 6/6, d1=0.320, d2=0.000 g=14.111
dataset.shape[0] = 50, n_samples = 10
1/1 [==============================] - 0s 23ms/step


>Accuracy real: 90%, fake: 100%
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 19ms/step
>175, 1/6, d1=0.205, d2=0.000 g=9.163
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 42ms/step
>175, 2/6, d1=0.023, d2=0.009 g=4.863
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 27ms/step
>175, 3/6, d1=0.155, d2=0.724 g=7.948
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 31ms/step
>175, 4/6, d1=0.058, d2=0.001 g=9.533
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 22ms/step
>175, 5/6, d1=0.557, d2=0.073 g=3.403
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 20ms/step
>175, 6/6, d1=0.029, d2=1.614 g=12.897
dataset.shape[0] = 50, n_samples = 10
1/1 [==============================] - 0s 19ms/step


...

>Accuracy real: 100%, fake: 100%
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 17ms/step
>197, 1/6, d1=0.000, d2=0.000 g=15.138
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 22ms/step
>197, 2/6, d1=0.000, d2=0.000 g=15.119
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 22ms/step
>197, 3/6, d1=0.000, d2=0.000 g=14.930
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 29ms/step
>197, 4/6, d1=0.000, d2=0.000 g=15.056
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 49ms/step
>197, 5/6, d1=0.000, d2=0.000 g=14.927
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 44ms/step
>197, 6/6, d1=0.000, d2=0.000 g=15.022
dataset.shape[0] = 50, n_samples = 10
1/1 [==============================] - 0s 35ms/step


>Accuracy real: 100%, fake: 100%
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 34ms/step
>198, 1/6, d1=0.000, d2=0.000 g=14.920
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 35ms/step
>198, 2/6, d1=0.000, d2=0.000 g=15.035
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 22ms/step
>198, 3/6, d1=0.000, d2=0.000 g=15.094
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 32ms/step
>198, 4/6, d1=0.000, d2=0.000 g=14.985
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 25ms/step
>198, 5/6, d1=0.000, d2=0.000 g=15.072
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 22ms/step
>198, 6/6, d1=0.000, d2=0.000 g=14.983
dataset.shape[0] = 50, n_samples = 10
1/1 [==============================] - 0s 20ms/step


>Accuracy real: 100%, fake: 100%
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 22ms/step
>199, 1/6, d1=0.000, d2=0.000 g=15.000
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 22ms/step
>199, 2/6, d1=0.000, d2=0.000 g=15.044
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 22ms/step
>199, 3/6, d1=0.000, d2=0.000 g=15.175
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 21ms/step
>199, 4/6, d1=0.000, d2=0.000 g=14.896
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 23ms/step
>199, 5/6, d1=0.000, d2=0.000 g=15.109
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 22ms/step
>199, 6/6, d1=0.000, d2=0.000 g=15.105
dataset.shape[0] = 50, n_samples = 10
1/1 [==============================] - 0s 21ms/step


>Accuracy real: 100%, fake: 100%
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 21ms/step
>200, 1/6, d1=0.000, d2=0.000 g=15.044
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 23ms/step
>200, 2/6, d1=0.000, d2=0.000 g=15.055
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 22ms/step
>200, 3/6, d1=0.000, d2=0.000 g=15.161
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 23ms/step
>200, 4/6, d1=0.000, d2=0.000 g=15.086
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 21ms/step
>200, 5/6, d1=0.000, d2=0.000 g=15.251
dataset.shape[0] = 50, n_samples = 4
1/1 [==============================] - 0s 21ms/step
>200, 6/6, d1=0.000, d2=0.000 g=15.246
dataset.shape[0] = 50, n_samples = 10
1/1 [==============================] - 0s 20ms/step


>Accuracy real: 100%, fake: 100%


# Generation of new CGR images

In [7]:
from keras.models import load_model
from numpy import asarray
from PIL import Image
import random
import numpy as np
import keras
import os

mainPath = '/content/drive/MyDrive/ML_Dataset'
augmentedPath = mainPath + '/AugmentedModels'
classNames = os.listdir(augmentedPath)

# load model

for classID in range(0, 1):
    print('Class', classNames[classID])
    for modelNo in range(171, 201):
        model = load_model(augmentedPath +"/" + classNames[classID] + '/generator_class' + str(8) + '_model_' + str(modelNo) + '.h5')
        outputFolder = '/content/drive/MyDrive/ML_Dataset/Augmented/Model' + str(modelNo) + '/' + classNames[classID] + '/'
        try:#create output folder
            os.makedirs(outputFolder)
        except OSError:
            pass

        noOfAugmentedImages = 1

        for j in range(noOfAugmentedImages):
            vector = []
            for i in range(100):
                vector.append(random.uniform(0, 1))
            # generate image
            X = model.predict(asarray([vector]))

            # scale from [-1,1] to [0,1]
            X = (X + 1) / 2.0
            Y = (X[0] * 255).astype(np.uint8)

            pil_img = Image.fromarray(Y)
            pil_img.save(outputFolder + 'Augmented' + str(j) + 'Class' + str(classID) + 'Model' + str(modelNo) + '.jpg')
        keras.backend.clear_session()

Class RobinSidel


1/1 [==============================] - 0s 171ms/step


1/1 [==============================] - 0s 147ms/step


1/1 [==============================] - 0s 144ms/step


1/1 [==============================] - 0s 141ms/step


...

1/1 [==============================] - 0s 144ms/step


1/1 [==============================] - 0s 134ms/step


1/1 [==============================] - 0s 138ms/step


1/1 [==============================] - 0s 142ms/step


1/1 [==============================] - 0s 131ms/step
